# How to make targeted offers to customers

This tutorial includes everything you need to set up IBM Decision Optimization CPLEX Modeling for Python (DOcplex), build a Mathematical Programming model, and get its solution by solving the model with IBM ILOG CPLEX Optimizer.

Table of contents:

-  [Describe the business problem](#Describe-the-business-problem)
*  [How decision optimization (prescriptive analytics) can help](#How-decision-optimization-can-help)
*  [Prepare the data](#Prepare-the-data)
*  [Use decision optimization](#Use-IBM-Decision-Optimization-CPLEX-Modeling-for-Python)
    -  [Step 1: Set up the prescriptive model](#Step-1:-Set-up-the-prescriptive-model)
        * [Define the decision variables](#Define-the-decision-variables)
        * [Set up the constraints](#Set-up-the-constraints)
        * [Express the objective](#Express-the-objective)
        * [Solve with Decision Optimization](#Solve-with-Decision-Optimization)
    *  [Step 2: Analyze the solution](#Step-2:-Analyze-the-solution)


## Describe the business problem

* This example is based on a fictional banking company. 
* The marketing department wants to achieve more profitable results in future campaigns by matching the right offer of financial services to each customer. 
* In particular, the data science department identified the characteristics of customers who are most likely to respond favorably based on previous offers and responses and to promote the best current offer based on the results and now need to compute the best offering plan.
<br>

A set of business constraints have to be respected:

* You have a limited budget to run a marketing campaign based on "gifts", "newsletter", "seminar"...
* You want to determine which is the best way to contact the customers.
* You need to identify which customers to contact.

<a id="#How-decision-optimization-can-help"></a>
## How decision optimization can help

* Prescriptive analytics technology recommends actions based on desired outcomes, taking into account specific scenarios, resources, and knowledge of past and current events. This insight can help your organization make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

+ For example:
    + Automate complex decisions and trade-offs to better manage limited resources.
    + Take advantage of a future opportunity or mitigate a future risk.
    + Proactively update recommendations based on changing events.
    + Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.

<a id="#Prepare-the-data"></a>

## Prepare the data

In [1]:
# The code was removed by Watson Studio for sharing.

,age,income,members_in_household,loan_accounts,Savings,Mortgage,Pension,id,nb_products
0,38,47958.0,4,1,0,0,0,44256,0
1,30,48606.0,2,4,0,0,0,46883,0
2,41,42152.0,4,0,0,0,0,32387,0
3,42,39788.0,3,3,0,0,0,25504,0
4,42,44365.0,6,2,0,1,0,35979,1


In [2]:
offers = df_data_1.loc[df_data_1['nb_products'] >= 1]

### Assign campaigns to customers
You have a prediction of who will buy what in the list of new customers.
However, you do not have the budget to contact all of them. You have various contact channels with different costs and effectiveness.
Furthermore, if you contact a customer you want to propose only one product per customer.
Some input data for optimization

In [3]:
# How much revenue is earned when selling each product
products = ["Savings", "Mortgage", "Pension"]
productValue = [200, 300, 400]
value_per_product = {products[i] : productValue[i] for i in range(len(products))}

# Total available budget
availableBudget = 25000

# For each channel, cost of making a marketing action and success factor
channels =  pd.DataFrame(data=[("gift", 20.0, 0.20), 
                               ("newsletter", 15.0, 0.05), 
                               ("seminar", 23.0, 0.30)], columns=["name", "cost", "factor"])

offersR = range(0, len(offers))
productsR = range(0, len(products))
channelsR = range(0, len(channels))

## Use IBM Decision Optimization CPLEX Modeling for Python

Create the optimization model to select the best ways to contact customers and stay within the limited budget.

### Step 1: Set up the prescriptive model

Set up the prescriptive model using the Mathematical Programming (docplex.mp) modeling package. 

#### Create the model

In [4]:
from docplex.mp.model import Model

mdl = Model(name="marketing_campaign")

#### Define the decision variables
- The integer decision variables `channelVars`, represent whether or not a customer will be made an offer for a particular product via a particular channel.
- The integer decision variable `totaloffers` represents the total number of offers made.
- The continuous variable `budgetSpent` represents the total cost of the offers made.

In [5]:
channelVars = mdl.binary_var_cube(offersR, productsR, channelsR)
totaloffers = mdl.integer_var(lb=0)
budgetSpent = mdl.continuous_var()

#### Set up the constraints
- Offer only one product per customer.
- Compute the budget and set a maximum for it.
- Compute the number of offers to be made.

In [6]:
# At most 1 product is offered to each customer
mdl.add_constraints( mdl.sum(channelVars[o,p,c] for p in productsR for c in channelsR) <=1
                   for o in offersR)

# Do not exceed the budget
mdl.add_constraint( mdl.sum(channelVars[o,p,c]*channels.at[c, "cost"]
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR)  <= availableBudget, "budget")  

# At least 10% offers per channel
for c in channelsR:
    mdl.add_constraint(mdl.sum(channelVars[o,p,c] for p in productsR for o in offersR) >= len(offers) // 10)

mdl.print_information()

Model: marketing_campaign
 - number of variables: 10109
   - binary=10107, integer=1, continuous=1
 - number of constraints: 1127
   - linear=1127
 - parameters: defaults
 - problem type is: MILP


In [7]:
offers.reset_index(inplace=True)

#### Express the objective

Maximize the expected revenue.

In [8]:
obj = 0

for c in channelsR:
    for p in productsR:
        product=products[p]
        coef = channels.at[c,"factor"] * value_per_product[product]
        obj += mdl.sum(channelVars[o,p,c] * coef* offers.at[o, product] for o in offersR)

mdl.maximize(obj)

#### Solve with Decision Optimization

Depending on the size of the problem, the solve stage might fail and require the Commercial Edition of CPLEX engines, which is included in the premium environments in Watson Studio.

In [9]:
s = mdl.solve()
assert s, "No Solution !!!"

In [10]:
print(mdl.get_solve_status())
print(mdl.get_solve_details())

JobSolveStatus.OPTIMAL_SOLUTION
status  = integer optimal solution
time    = 0.112909 s.
problem = MILP
gap     = 0%



### Step-2: Analyze the solution

First, display the **Optimal Marketing Channel per customer**.

In [11]:
totaloffers = mdl.sum(channelVars[o,p,c] 
                      for o in offersR
                      for p in productsR 
                      for c in channelsR)

mdl.add_kpi(totaloffers, "nb offers")

budgetSpent = mdl.sum(channelVars[o,p,c]*channels.at[c,"cost"]
                                           for o in offersR 
                                           for p in productsR 
                                           for c in channelsR)
mdl.add_kpi(budgetSpent, "budget Spent")

for c in channelsR:
    channel = channels.at[c,"name"]
    kpi = mdl.sum(channelVars[o,p,c] for p in productsR for o in offersR)
    mdl.add_kpi(kpi, channel)

for p in productsR:
    product = products[p]
    kpi = mdl.sum(channelVars[o,p,c] for c in channelsR for o in offersR)
    mdl.add_kpi(kpi, product)
    
mdl.report()

* model marketing_campaign solved with objective = 79280.000
*  KPI: nb offers    = 1123.000
*  KPI: budget Spent = 24597.000
*  KPI: gift         = 112.000
*  KPI: newsletter   = 112.000
*  KPI: seminar      = 899.000
*  KPI: Savings      = 607.000
*  KPI: Mortgage     = 374.000
*  KPI: Pension      = 142.000


In [12]:
report = [(offers.at[o, "id"],channels.at[n, "name"], products[p],channels.at[c, "cost"]) 
          for c in channelsR
          for n in channelsR
          for p in productsR 
          for o in offersR  if channelVars[o,p,c].solution_value==1]



print("Marketing plan has {0} offers costing {1}".format(totaloffers.solution_value, budgetSpent.solution_value))

report_bd = pd.DataFrame(report, columns=['customer_id','channel', 'product','cost'])
display(report_bd)

Marketing plan has 1123 offers costing 24597.0


,customer_id,channel,product,cost
0,55728,gift,Savings,20.0
1,94027,gift,Savings,20.0
2,56258,gift,Savings,20.0
3,58943,gift,Savings,20.0
4,49240,gift,Savings,20.0
5,49293,gift,Savings,20.0
6,58456,gift,Savings,20.0
7,60877,gift,Savings,20.0
8,73798,gift,Savings,20.0
9,52777,gift,Savings,20.0



## Summary


You have learned how to set up and use IBM Decision Optimization CPLEX Modeling for Python to formulate a Mathematical Programming model and solve it with CPLEX.


## References
* <a href="https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html" target="_blank" rel="noopener noreferrer">Decision Optimization CPLEX Modeling for Python documentation</a>
* <a href="https://dataplatform.cloud.ibm.com/docs/content/getting-started/welcome-main.html?audience=wdp&context=wdp" target="_blank" rel="noopener noreferrer">Watson Studio documentation</a>


<hr>
Copyright © 2017-2019. This notebook and its source code are released under the terms of the MIT License.